In [67]:
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.decomposition import PCA


 



### Load data from various directories

In [68]:
#combine all files into a single dataframe
def load_data(main_directory):
    subdir_data  = {}

    for sub_dir in os.listdir(main_directory):
        sub_path = os.path.join(main_directory, sub_dir)
        if os.path.isdir(sub_path): #make sure it is a directory
            csv_files = [os.path.join(sub_path,file) for file in os.listdir(sub_path) if file.endswith('.csv')] #creating filepaths to csvs

            if csv_files:
                try:
                    dataframes = [pd.read_csv(csv_file) for csv_file in csv_files] #make a list of dataframes
                    merged = pd.concat(dataframes, ignore_index=True) #combine all dataframes
                    subdir_data[sub_dir] = merged
                except Exception as e:
                    print(f"Error processing subdirectory {sub_path}: {e}")
    total_merged = pd.concat(subdir_data.values(), ignore_index=True if subdir_data else pd.DataFrame())
    
    return total_merged
#############################################################################################################################################################

run = 2
run_data = f"C:\\Github_Repos\\MATH2015-Linear-Regression-Model\\Linear_Regression_Data\\Season Datasets\\Run_{run}"
data = load_data(run_data)





In [69]:
#checking columns names
print(data.columns)

Index(['final_PTS', 'Court', 'TS%', 'FTr', 'USG%', 'PER', 'avg_PTS', 'FGA',
       'MP', 'ORtg', 'eFG%_2', 'Pace', 'oFG%', 'oFG% 3P', 'oAvg Distance',
       'oFGA 2P', 'oFGA 0-3', 'oFGA 3-10', 'oFGA 10-16', 'oFGA 16-3P',
       'oFGA 3P', 'oFG% 2P', 'oFG% 0-3', 'oFG% 3-10', 'oFG% 10-16',
       'oFG% 16-3P', 'FG%'],
      dtype='object')


### Feature Scaling

In [70]:
# Split the features up depending on how they are doing to be scaled


static_vars = ["TS%", "FTr", "USG%", "PER", "avg_PTS", "FGA", "MP", "ORtg", "eFG%_2", "Pace"]

dynamic_vars = ["oFG%","oFG% 3P","oAvg Distance","oFGA 2P","oFGA 0-3","oFGA 3-10", 
                "oFGA 10-16","oFGA 16-3P","oFGA 3P","oFG% 2P","oFG% 0-3","oFG% 3-10", 
                "oFG% 10-16","oFG% 16-3P","FG%"]


#Court is binary, will not scale

### Feature Scaling

In [71]:
#get target variables and features
y_pts = data['final_PTS']
x_feat = data.drop('final_PTS',axis=1)

#Look at variances 
variances = x_feat.var()

print('The feature variances for this run are:\n\n', variances)

#separate static and dynamic variables


scaler_MinMax = MinMaxScaler()
scaler_Zscore = StandardScaler()


x_feat[static_vars] = scaler_MinMax.fit_transform(X[static_vars])
x_feat[dynamic_vars] = scaler_Zscore.fit_transform(X[dynamic_vars])



The feature variances for this run are:

 Court             0.250002
TS%               0.006943
FTr               0.020281
USG%             25.781081
PER              23.585902
avg_PTS          33.090432
FGA              18.427881
MP               68.030587
ORtg             11.147624
eFG%_2            0.000228
Pace              3.113567
oFG%              0.000192
oFG% 3P           0.000146
oAvg Distance     0.278583
oFGA 2P           0.000642
oFGA 0-3          0.000974
oFGA 3-10         0.000904
oFGA 10-16        0.000141
oFGA 16-3P        0.000083
oFGA 3P           0.000642
oFG% 2P           0.000316
oFG% 0-3          0.000615
oFG% 3-10         0.000701
oFG% 10-16        0.000452
oFG% 16-3P        0.000689
FG%               0.000192
dtype: float64


In [72]:
# for column in x_feat.columns:
#     sns.scatterplot(x=x_feat[column], y=y_pts)
#     plt.title(f'Scatter plot: {column} vs target')
#     plt.show()

### Observe Multicollinearity  
Multicollinearity occurs when two or more features (independent variables) in a dataset are highly correlated, meaning they contain similar information about the variance in the target variable. This can lead to issues in statistical models, especially in linear regression, where the goal is to isolate the effect of each feature on the target.
 - Low: VIF < 5
 - Moderate: 5 < VIF < 10
 - Extremely high: 10 < VIF

 All features with VIF over 10 need to be either deleted or engineered



In [73]:

# Calculate VIF for each feature
vif_data = pd.DataFrame()
vif_data["feature"] = x_feat.columns
vif_data["VIF"] = [variance_inflation_factor(x_feat.values, i) for i in range(x_feat.shape[1])]
print(vif_data)




c:\Users\kings\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


          feature         VIF
0           Court    1.984850
1             TS%  110.582286
2             FTr    5.885689
3            USG%   33.663865
4             PER  156.890669
5         avg_PTS  103.883424
6             FGA  168.391399
7              MP   49.075839
8            ORtg   10.094545
9          eFG%_2    9.925383
10           Pace    5.388706
11           oFG%         inf
12        oFG% 3P  144.594506
13  oAvg Distance   59.648937
14        oFGA 2P         inf
15       oFGA 0-3  229.280707
16      oFGA 3-10  204.963598
17     oFGA 10-16   32.496207
18     oFGA 16-3P   23.342936
19        oFGA 3P         inf
20        oFG% 2P  864.421879
21       oFG% 0-3   56.040679
22      oFG% 3-10   48.712324
23     oFG% 10-16    7.001857
24     oFG% 16-3P    5.960712
25            FG%         inf


In [74]:
# Compute correlation matrix
correlation_matrix = x_feat.corr()

# Identify features with high correlation
high_corr_features = [column for column in correlation_matrix.columns if any(correlation_matrix[column] > 0.9)]
print(high_corr_features)


['Court', 'TS%', 'FTr', 'USG%', 'PER', 'avg_PTS', 'FGA', 'MP', 'ORtg', 'eFG%_2', 'Pace', 'oFG%', 'oFG% 3P', 'oAvg Distance', 'oFGA 2P', 'oFGA 0-3', 'oFGA 3-10', 'oFGA 10-16', 'oFGA 16-3P', 'oFGA 3P', 'oFG% 2P', 'oFG% 0-3', 'oFG% 3-10', 'oFG% 10-16', 'oFG% 16-3P', 'FG%']


In [75]:
## THIS MADE THE MODEL PERFORM MUCH WORSE

pca = PCA()
pca.fit(x_feat)
cumulative_variance = pca.explained_variance_ratio_.cumsum()
# Find the number of components to retain 95% variance
n_components = next(i for i, total in enumerate(cumulative_variance) if total >= 0.95) + 1
print("Number of components to retain 95% variance:", n_components)

pca = PCA(n_components=n_components)
x_feat_pca = pca.fit_transform(x_feat)

Number of components to retain 95% variance: 9


### Model Fitting

In [76]:

#split the data into training and testing sets: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(x_feat, y_pts, test_size=0.2, random_state=42) #rand state is the seed, ensures that same testing and training data is created

model = LinearRegression()

model.fit(X_train, y_train)

LinearRegression()

### Model Evaluation

**MSE** measures the average squared difference between predicted and actual values. It's in the squared units of your target variable.

**RMSE** is the square root of MSE, making it interpretable in the same units as the target variable.

**R^2** represents the proportion of the variance in the target variable that is explained by the model.
 - R^2=1 is a perfect fit
 - R^2 = 0 meas the model predicts no better than the mean of target variable
 - R^2 < 0 means the model is worse than the mean for prediction


In [77]:
y_predict = model.predict(X_test)
mse = mean_squared_error(y_test, y_predict)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,y_predict)

print(f"Test Mean Squared Error: {mse}")
print(f"Test RMSE: {rmse}") #this is the average points we are off by in predictions
print(f"Test R² Score: {r2}")

Test Mean Squared Error: 42.43630652111648
Test RMSE: 6.514315506721829
Test R² Score: 0.31941928466351854
